<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG19 Training

This notebook is used to train VGG19 model for glacoma diagnosis.

In [1]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     !pip install deriva
#     !pip install bdbag
#     !pip install --upgrade --force pydantic
#     !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [2]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [3]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [4]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2024-06-26 06:37:01,683 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-26 06:37:01,684 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [5]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid = "2-C94P" # rid
# Change the confi_file with bag_url=["minid: train", "minid: Valid", "minid: test"]


In [6]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2024-06-26 06:37:01,727 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-26 06:37:01,728 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [7]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

2024-06-26 06:37:03,524 - INFO - File [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_training_sreenidhi_june_24_2024.json] transfer successful. 0.87 KB transferred. Elapsed time: 0:00:00.000075.
2024-06-26 06:37:03,525 - INFO - Verifying MD5 checksum for downloaded file [/data/sreenidhi/EyeAI_working/Execution_Metadata/Execution_Config-vgg19_catalog_model_training_sreenidhi_june_24_2024.json]
2024-06-26 06:37:03,545 - INFO - Configuration validation successful!
2024-06-26 06:37:14,024 - INFO - File [/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json] transfer successful. 0.69 KB transferred. Elapsed time: 0:00:00.000054.
2024-06-26 06:37:14,025 - INFO - Verifying SHA256 checksum for downloaded file [/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json]
2024-06-26 06:37:14,296 - INFO - File [/data/sreenidhi/EyeAI_working/

{'caching_dir': PosixPath('/data'),
 'working_dir': PosixPath('/data/sreenidhi/EyeAI_working'),
 'vocabs': {'Workflow_Type': [{'name': 'VGG19_Catalog_Model_training',
    'rid': '2-A7D8'}],
  'Execution_Asset_Type': [{'name': 'VGG19_Catalog_Model', 'rid': '2-A7DA'}]},
 'execution_rid': '2-C6CM',
 'workflow_rid': '2-A7CR',
 'bag_paths': [PosixPath('/data/2-277G_6aa1a6861eee5a79bce4bf071065355f95a066c2a1ff326089d43048a7e0f185/Dataset_2-277G'),
  PosixPath('/data/2-277J_81c873a311aa6a67cf2eef44bd9056cb19181b299a6e44327ea3553616f18725/Dataset_2-277J'),
  PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')],
 'assets_paths': [PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json'),
  PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/train_no_optic_disc_image_ids.csv'),
  PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/valid_no_optic_disc_image_ids.

In [8]:
configuration_records

ConfigurationRecord(caching_dir=PosixPath('/data'), working_dir=PosixPath('/data/sreenidhi/EyeAI_working'), vocabs={'Workflow_Type': [Term(name='VGG19_Catalog_Model_training', rid='2-A7D8')], 'Execution_Asset_Type': [Term(name='VGG19_Catalog_Model', rid='2-A7DA')]}, execution_rid='2-C6CM', workflow_rid='2-A7CR', bag_paths=[PosixPath('/data/2-277G_6aa1a6861eee5a79bce4bf071065355f95a066c2a1ff326089d43048a7e0f185/Dataset_2-277G'), PosixPath('/data/2-277J_81c873a311aa6a67cf2eef44bd9056cb19181b299a6e44327ea3553616f18725/Dataset_2-277J'), PosixPath('/data/2-277M_8c4b855c2752e098580a5bb0d1b63a8cedde4462805fe74cddc912a72fb39963/Dataset_2-277M')], assets_paths=[PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json'), PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/train_no_optic_disc_image_ids.csv'), PosixPath('/data/sreenidhi/EyeAI_working/Execution_Assets/valid_no_optic_disc_image_ids.csv')], configuration_path

In [9]:
exclude_train = pd.read_csv(configuration_records.assets_paths[1])['ID'].to_list()
exclude_valid = pd.read_csv(configuration_records.assets_paths[2])['ID'].to_list()

In [11]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
train_dir = configuration_records.bag_paths[0] # path to the raw train dataset
validation_dir = configuration_records.bag_paths[1]
test_dir = configuration_records.bag_paths[2]

train_cropped_image_path, train_cropped_csv = EA.create_cropped_images(str(train_dir),
                                                                       output_dir = str(EA.working_dir) +'/train',
                                                                       crop_to_eye=True,
                                                                       exclude_list=exclude_train)
validation_cropped_image_path, validation_cropped_csv = EA.create_cropped_images(str(validation_dir),
                                                                                 output_dir = str(EA.working_dir) +'/valid',
                                                                                 crop_to_eye=True,
                                                                                 exclude_list=exclude_valid)
test_cropped_image_path, test_cropped_csv = EA.create_cropped_images(str(test_dir),
                                                                     output_dir = str(EA.working_dir) +'/test',
                                                                     crop_to_eye=True)


In [12]:
# # without no optic disc images

import os

def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

def analyze_directory(base_path):
    main_folders = ['train', 'test', 'valid']
    
    for main_folder in main_folders:
        main_folder_path = os.path.join(base_path, main_folder)
        if not os.path.exists(main_folder_path):
            print(f"{main_folder} folder not found")
            continue
        
        print(f"\nAnalyzing {main_folder} folder:")
        
        image_cropped_path = os.path.join(main_folder_path, 'Image_cropped')
        if not os.path.exists(image_cropped_path):
            print("Image_cropped folder not found")
            continue
        
        total_files = 0
        for subfolder in os.listdir(image_cropped_path):
            subfolder_path = os.path.join(image_cropped_path, subfolder)
            if os.path.isdir(subfolder_path):
                file_count = count_files(subfolder_path)
                print(f"  {subfolder}: {file_count} files")
                total_files += file_count
        
        print(f"Total files in {main_folder}: {total_files}")

# Assuming you're running this script from the directory containing train, test, and valid folders
base_path = "/data/sreenidhi/EyeAI_working/" #os.getcwd()
analyze_directory(base_path)


Analyzing train folder:
  2SKC_No_Glaucoma: 5490 files
  2SKA_Suspected_Glaucoma: 3506 files
Total files in train: 8996

Analyzing test folder:
  2SKC_No_Glaucoma: 526 files
  2SKA_Suspected_Glaucoma: 568 files
Total files in test: 1094

Analyzing valid folder:
  2SKC_No_Glaucoma: 1840 files
  2SKA_Suspected_Glaucoma: 1162 files
Total files in valid: 3002


In [18]:
# before with no optic disc images
import os

def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

def analyze_directory(base_path):
    main_folders = ['train', 'test', 'valid']
    
    for main_folder in main_folders:
        main_folder_path = os.path.join(base_path, main_folder)
        if not os.path.exists(main_folder_path):
            print(f"{main_folder} folder not found")
            continue
        
        print(f"\nAnalyzing {main_folder} folder:")
        
        image_cropped_path = os.path.join(main_folder_path, 'Image_cropped')
        if not os.path.exists(image_cropped_path):
            print("Image_cropped folder not found")
            continue
        
        total_files = 0
        for subfolder in os.listdir(image_cropped_path):
            subfolder_path = os.path.join(image_cropped_path, subfolder)
            if os.path.isdir(subfolder_path):
                file_count = count_files(subfolder_path)
                print(f"  {subfolder}: {file_count} files")
                total_files += file_count
        
        print(f"Total files in {main_folder}: {total_files}")

# Assuming you're running this script from the directory containing train, test, and valid folders
base_path = "/data/sreenidhi/EyeAI_working/" #os.getcwd()
analyze_directory(base_path)


Analyzing train folder:
  2SKC_No_Glaucoma: 5525 files
  2SKA_Suspected_Glaucoma: 3540 files
Total files in train: 9065

Analyzing test folder:
  2SKC_No_Glaucoma: 526 files
  2SKA_Suspected_Glaucoma: 568 files
Total files in test: 1094

Analyzing valid folder:
  2SKC_No_Glaucoma: 1860 files
  2SKA_Suspected_Glaucoma: 1173 files
Total files in valid: 3033


In [13]:

output_path = str(EA.working_dir) + "/Execution_Assets/" + configuration_records.vocabs['Execution_Asset_Type'][0].name
os.mkdir(output_path)

In [14]:
output_path

'/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_Model'

In [15]:
best_hyper_parameters_json_path = str(configuration_records.assets_paths[0])

In [16]:
best_hyper_parameters_json_path

'/data/sreenidhi/EyeAI_working/Execution_Assets/best_hyperparameters_exluding_no_optic_disc_images_june_24_2024.json'

In [17]:
# @title Execute Training algorithm
from eye_ai.models.vgg19_diagnosis_train import main
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  main(train_path=train_cropped_image_path,
       valid_path=validation_cropped_image_path, 
       test_path=test_cropped_image_path, 
       output_path = output_path,
       best_hyperparameters_json_path = best_hyper_parameters_json_path,
       model_name = "VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024"
       )
                    


2024-06-26 07:04:46.580653: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 07:04:46.580712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 07:04:46.717774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-26 07:04:46.993452: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 07:04:48.432593: W tensorflow/compiler/tf2

Found 8996 images belonging to 2 classes.
Found 3002 images belonging to 2 classes.
Found 1094 images belonging to 2 classes.
train_generator.class_indices :  {'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}
validation_generator.class_indices :  {'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}
test_generator.class_indices :  {'2SKC_No_Glaucoma': 0, '2SKA_Suspected_Glaucoma': 1}


2024-06-26 07:04:50.260668: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-26 07:04:50.720607: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-26 07:04:50.724838: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100


2024-06-26 07:04:55.939700: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-06-26 07:05:01.216789: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f34dcd11670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-26 07:05:01.216829: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A10G, Compute Capability 8.6
2024-06-26 07:05:01.242315: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1719410701.368375   10544 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


282/282 [==============================] - 88s 246ms/step - loss: 0.6169 - roc_auc_score: 0.7531 - f1_score_normal: 0.6313 - accuracy_score: 0.6865 - val_loss: 0.5790 - val_roc_auc_score: 0.8001 - val_f1_score_normal: 0.3782 - val_accuracy_score: 0.6875
Epoch 2/100
282/282 [==============================] - 73s 257ms/step - loss: 0.5130 - roc_auc_score: 0.8346 - f1_score_normal: 0.7053 - accuracy_score: 0.7539 - val_loss: 0.5677 - val_roc_auc_score: 0.8465 - val_f1_score_normal: 0.7131 - val_accuracy_score: 0.7335
Epoch 3/100
282/282 [==============================] - 68s 236ms/step - loss: 0.4683 - roc_auc_score: 0.8630 - f1_score_normal: 0.7403 - accuracy_score: 0.7863 - val_loss: 0.5852 - val_roc_auc_score: 0.8542 - val_f1_score_normal: 0.5294 - val_accuracy_score: 0.7425
Epoch 4/100
282/282 [==============================] - 72s 251ms/step - loss: 0.4490 - roc_auc_score: 0.8739 - f1_score_normal: 0.7514 - accuracy_score: 0.7967 - val_loss: 0.5048 - val_roc_auc_score: 0.8715 - val_f

2024-06-26 07:36:23,895 - INFO - Test results - [0.4728105068206787, 0.8924482464790344, 0.8003986477851868, 0.808043897151947]


Model Eval results: [0.4728105068206787, 0.8924482464790344, 0.8003986477851868, 0.808043897151947]


/home/sreenidhi/.conda/envs/my-tensorflow-conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2024-06-26 07:36:24,108 - INFO - VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024 Model trained, Model and training history are saved successfully.


In [18]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, True)

2024-06-26 07:36:25,719 - INFO - Initializing uploader: GenericUploader v1.7.1 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2024-06-26 07:36:25,721 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-06-26 07:36:25,721 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-06-26 07:36:25,760 - INFO - Checking for updated configuration...
2024-06-26 07:36:25,864 - INFO - Updated configuration found.
2024-06-26 07:36:25,866 - INFO - Scanning files in directory [/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_Model]...
2024-06-26 07:36:25,869 - INFO - Including file: [/data/sreenidhi/EyeAI_working/Execution_Assets/VGG19_Catalog_Model/VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024.h5].
2024-06-26 07:36:25,